<a href="https://colab.research.google.com/github/rkuo2000/AI-stocks/blob/main/ch06_stock_bot_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CH-06 個股分析機器人

### 1️⃣ 安裝及匯入套件

In [1]:
!pip install openai
!pip install yfinance

In [2]:
from openai import OpenAI, OpenAIError # 串接 OpenAI API
import yfinance as yf
import pandas as pd # 資料處理套件
import numpy as np
import datetime as dt # 時間套件
import requests
from bs4 import BeautifulSoup

### 2️⃣ 輸入 Gemini API KEY

In [3]:
from google.colab import userdata

client = OpenAI(
    api_key=userdata.get('GEMINI_API_KEY'),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

### 3️⃣ 取得股價資料

In [4]:
# 從 yfinance 取得一周股價資料
def stock_price(stock_id="大盤", days = 10):
  if stock_id == "大盤":
    stock_id="^TWII"
  else:
    stock_id += ".TW"

  end = dt.date.today() # 資料結束時間
  start = end - dt.timedelta(days=days) # 資料開始時間
  # 下載資料
  #df = yf.download(stock_id, start=start)
  df = yf.Ticker(stock_id).history(start=start)

  # 更換列名
  df.columns = ['日期','開盤價', '最高價', '最低價',
                '收盤價', '調整後收盤價', '成交量']

  data = {
    '日期': df.index.strftime('%Y-%m-%d').tolist(),
    '收盤價': df['收盤價'].tolist(),
    '每日報酬': df['收盤價'].pct_change().tolist(),
    '漲跌價差': df['調整後收盤價'].diff().tolist()
    }

  return data

print(stock_price("2330"))

{'日期': ['2025-10-08', '2025-10-09', '2025-10-13', '2025-10-14', '2025-10-15', '2025-10-16', '2025-10-17'], '收盤價': [23064494, 39052134, 53850992, 39889256, 41056405, 37737678, 38077331], '每日報酬': [nan, 0.6931710706508454, 0.37895132696205547, -0.2592660874288073, 0.02925973349816302, -0.08083335596480012, 0.009000368279150628], '漲跌價差': [nan, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}


### 4️⃣ 取得基本面資料

In [5]:
# 基本面資料
def stock_fundamental(stock_id= "大盤"):
  if stock_id == "大盤":
      return None

  stock_id += ".TW"
  stock = yf.Ticker(stock_id)

  # 營收成長率
  quarterly_revenue_growth = np.round(stock.quarterly_financials.loc["Total Revenue"].pct_change(-1).dropna().tolist(), 2)

  # 每季EPS
  quarterly_eps = np.round(stock.quarterly_financials.loc["Basic EPS"].dropna().tolist(), 2)

  # EPS季增率
  quarterly_eps_growth = np.round(stock.quarterly_financials.loc["Basic EPS"].pct_change(-1).dropna().tolist(), 2)

  # 轉換日期
  dates = [date.strftime('%Y-%m-%d') for date in stock.quarterly_financials.columns]

  data = {
      '季日期': dates[:len(quarterly_revenue_growth)],
      '營收成長率': quarterly_revenue_growth.tolist(),
      'EPS': quarterly_eps[0:3].tolist(),
      'EPS 季增率': quarterly_eps_growth[0:3].tolist()
  }

  return data

print(stock_fundamental("2330"))

{'季日期': ['2025-06-30', '2025-03-31', '2024-12-31', '2024-09-30', '2024-06-30'], '營收成長率': [0.11, -0.03, 0.14, 0.13, 0.0], 'EPS': [15.36, 13.94, 14.45], 'EPS 季增率': [0.1, -0.04, 0.15]}


/tmp/ipython-input-557545184.py:10: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  quarterly_revenue_growth = np.round(stock.quarterly_financials.loc["Total Revenue"].pct_change(-1).dropna().tolist(), 2)
/tmp/ipython-input-557545184.py:16: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  quarterly_eps_growth = np.round(stock.quarterly_financials.loc["Basic EPS"].pct_change(-1).dropna().tolist(), 2)


### 5️⃣ 取得新聞資料

In [6]:
# 新聞資料
def stock_news(stock_name ="大盤"):
  if stock_name == "大盤":
    stock_name="台股 -盤中速報"

  data=[]
  # 取得 Json 格式資料
  json_data = requests.get(f'https://ess.api.cnyes.com/ess/api/v1/news/keyword?q={stock_name}&limit=5&page=1').json()

  # 依照格式擷取資料
  items=json_data['data']['items']
  for item in items:
      # 網址、標題和日期
      news_id = item["newsId"]
      title = item["title"]
      publish_at = item["publishAt"]
      # 使用 UTC 時間格式
      utc_time = dt.datetime.fromtimestamp(publish_at, dt.timezone.utc)
      formatted_date = utc_time.strftime('%Y-%m-%d')
      # 前往網址擷取內容
      url = requests.get(f'https://news.cnyes.com/'
                        f'news/id/{news_id}').content
      soup = BeautifulSoup(url, 'html.parser')
      p_elements=soup .find_all('p')
      # 提取段落内容
      p=''
      for paragraph in p_elements[4:]:
          p+=paragraph.get_text()
      data.append([stock_name, formatted_date ,title,p])
  return data

print(stock_news("台積電"))

[['台積電', '2025-10-18', 'AI黃金週期進行式，從算力戰到能源戰的產業新格局： <mark>台積電</mark>、雙鴻、華城、中興電', '2025 年全球 AI 伺服器出貨量預估達 181 萬台，年增率高達 40%，搭載 HBM 高頻寬記憶體的高階伺服器將突破百萬台，北美四大 CSP 微軟、Google、亞馬遜、Meta 仍是主導者，微軟採購成長預計近八成，而甲骨文與 OpenAI 合作星際之門（Stargate Project）後，成為出貨增幅最快的雲端業者。台廠如 光寶科 (2301-TW)、台達電(2308-TW)、建準(2421-TW)、雙鴻(3324-TW)、緯穎(6669-TW) 及雲達，在 OCP 全球峰會上展示完整 AI 硬體生態鏈實力，涵蓋電源模組、液冷系統與整機 ODM 設計，隨 AI 資本支出擴增，台灣 ODM 廠正從代工製造升級為技術夥伴，迎接長期結構轉型。〈先進製程三強鼎立，CoWoS 產能持續加碼〉晶圓代工市場今年預估營收成長 17%，達 1650 億美元，先進製程與高階封裝技術是關鍵驅動力，2 奈米節點競賽持續升溫，台積電 (2330-TW)\xa0以量產進度與客戶黏著度穩居領先，三星與 Intel 則積極擴大 GAA 架構與先進封裝投資。後段封裝成為焦點，2.5D/3.5D 堆疊、HBM 整合與 Chiplet 設計為主流趨勢，推升 AI 晶片效能，台積電預計 2026 年前將 CoWoS 月產能由 9.5 萬片增至 11 萬片，以支應輝達、博通等長約訂單，未來，先進製程與封裝整合力將決定代工市場「第二戰場」勝負。〈立即填表體驗諮詢陳智霖分析師會員服務〉忠實粉絲務必先完成填表申請，立即體驗每週精選操作名單、盤中到價通知與即時策略更新，立即填表跟上專業：https://lihi.cc/RFzlE〈SMR 小型核能成 AI 算力基石 推動跨產業共演〉AI 算力需求暴增，使能源供應成為戰略焦點，美國能源部重申 2035 年前新增 35GW 核能產能、2040 年起每年部署 15GW 的長期計畫，SMR（小型模組化反應爐）被視為 AI 資料中心友善型能源。SMR 具模組化與快速建置優勢，美國 BWXT、NuScale 已啟動商轉，亞洲則以中國與韓國為主導，SMR 小型化可補足再生能源間歇性，為 AI 產業提供穩定低碳電力

### 6️⃣ 爬取股號、股名對照表

In [7]:
# 取得全部股票的股號、股名
def stock_name():
  print("線上讀取股號、股名、及產業別")

  response = requests.get('https://isin.twse.com.tw/isin/C_public.jsp?strMode=2')
  url_data = BeautifulSoup(response.text, 'html.parser')
  stock_company = url_data.find_all('tr')

  # 資料處理
  data = [
      (row.find_all('td')[0].text.split('\u3000')[0].strip(),
        row.find_all('td')[0].text.split('\u3000')[1],
        row.find_all('td')[4].text.strip())
      for row in stock_company[2:] if len(row.find_all('td')[0].text.split('\u3000')[0].strip()) == 4
  ]

  df = pd.DataFrame(data, columns=['股號', '股名', '產業別'])

  return df

name_df = stock_name()

線上讀取股號、股名、及產業別


### 7️⃣ 取得股票名稱

In [8]:
# 取得股票名稱
def get_stock_name(stock_id, name_df):
    return name_df.set_index('股號').loc[stock_id, '股名']

print(name_df.head())
print("--------------------------")
print(get_stock_name("1417",name_df))

     股號  股名   產業別
0  1101  台泥  水泥工業
1  1102  亞泥  水泥工業
2  1103  嘉泥  水泥工業
3  1104  環泥  水泥工業
4  1108  幸福  水泥工業
--------------------------
嘉裕


### 8️⃣ 建構 Gemini 2.5 Flash 模型

In [9]:
# 建立 Gemini 2.5 Flash 模型
def get_reply(messages):
    try:
        response = client.chat.completions.create(
            model = "gemini-2.5-flash",
            messages = messages
        )
        reply = response.choices[0].message.content
    except OpenAIError as err:
        reply = f"發生 {err.type} 錯誤\n{err.message}"
    return reply

# 建立訊息指令(Prompt)
def generate_content_msg(stock_id, name_df):

    stock_name = get_stock_name(
        stock_id, name_df) if stock_id != "大盤" else stock_id

    price_data = stock_price(stock_id)
    news_data = stock_news(stock_name)

    content_msg = f'請依據以下資料來進行分析並給出一份完整的分析報告:\n'

    content_msg += f'近期價格資訊:\n {price_data}\n'

    if stock_id != "大盤":
        stock_value_data = stock_fundamental(stock_id)
        content_msg += f'每季營收資訊：\n {stock_value_data}\n'

    content_msg += f'近期新聞資訊: \n {news_data}\n'
    content_msg += f'請給我{stock_name}近期的趨勢報告,請以詳細、\
      嚴謹及專業的角度撰寫此報告,並提及重要的數字, reply in 繁體中文'

    return content_msg

# StockGPT
def stock_gpt(stock_id, name_df=name_df):
    content_msg = generate_content_msg(stock_id, name_df)

    msg = [{
        "role": "system",
        "content": f"你現在是一位專業的證券分析師, 你會統整近期的股價\
      、基本面、新聞資訊等方面並進行分析, 然後生成一份專業的趨勢分析報告"
    }, {
        "role": "user",
        "content": content_msg
    }]

    reply_data = get_reply(msg)

    return reply_data


### 9️⃣ 大盤趨勢報告

In [10]:
reply = stock_gpt(stock_id="大盤")
print(reply)

## 台灣股市近期趨勢分析報告

**報告日期：** 2025年10月17日

**研究目的：** 本報告旨在統整2025年10月上旬台灣股市的價格資訊、近期新聞動態及總體經濟背景，以專業且嚴謹的角度分析市場趨勢，並提出相應的投資展望與建議。

---

### 一、 近期市場表現回顧

鑑於所提供之收盤價數據呈現極高的金額與波動性（單日漲跌幅可達±30%以上），此可能代表特定高價值資產、基金淨值或經調整後之指數參考點，而非一般上市股票價格。本報告將其視為市場整體情緒或特定趨勢之強烈指標進行分析。

**1. 價格數據分析 (2025年10月08日 – 2025年10月17日):**
*   **起始與結束價位：** 從2025年10月8日的4,475,000點至10月17日的4,488,400點，期間總體漲幅約為0.30%，顯示此期間市場表現相對持平，但其間的波動極為劇烈。
*   **劇烈波動性：**
    *   **高點：** 10月14日達到6,407,700點的近期高點。
    *   **低點：** 10月15日則迅速回落至4,438,300點。
    *   **單日報酬：** 市場經歷了數次大幅度的單日漲跌：
        *   10月9日：上漲24.05%。
        *   10月13日：下跌12.67%。
        *   10月14日：再次飆升32.17%。
        *   10月15日：隨後重挫30.73%。
        *   最近兩個交易日 (10/16-10/17) 波動趨緩，分別上漲4.95%與下跌3.64%。
*   **漲跌價差：** 提供的數據中「漲跌價差」皆為0.0，此資訊可能不適用於當前分析，或表示無顯著的開盤跳空情況。

**小結：** 近期市場呈現極端的高波動性，雖然起始與結束價位變化不大，但其間的劇烈震盪顯示投資者情緒極不穩定，多空雙方拉鋸激烈，市場存在高度不確定性。

### 二、 關鍵新聞事件與影響分析

近期新聞主要圍繞在總體經濟政策、半導體及AI產業的展望，以及法人籌碼動向。

**1. 總體經濟與貨幣政策：**
*   **聯準會降息循環：** 最新消息指出美國聯準會（Fed）已開啟降息循環，2025年9月聯邦基準利率降至4.00%-4.25%，並預估年底前還會再降息2碼

### 🔟 個股分析報告

In [11]:
reply = stock_gpt(stock_id="2330")
print(reply)

/tmp/ipython-input-557545184.py:10: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  quarterly_revenue_growth = np.round(stock.quarterly_financials.loc["Total Revenue"].pct_change(-1).dropna().tolist(), 2)
/tmp/ipython-input-557545184.py:16: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  quarterly_eps_growth = np.round(stock.quarterly_financials.loc["Basic EPS"].pct_change(-1).dropna().tolist(), 2)


## 台積電 (TSMC) 近期趨勢分析報告

**報告日期：** 2025年10月18日
**公司名稱：** 台灣積體電路製造股份有限公司 (Taiwan Semiconductor Manufacturing Company, TSMC)
**股票代號：** 2330-TW (或根據上下文判斷為其他市場代號)

---

### 一、 近期股價表現分析

**重要提示：**
所提供的股價資料顯示，收盤價數據異常龐大 (例如：23,064,494至38,077,331)，顯然不符合台積電實際股價水平，可能為數據輸入錯誤或非股價之其他識別碼。因此，無法依據此數據對台積電近期股價走勢進行實質性的量化分析。

**初步觀察：**
儘管原始股價數據存在可靠性問題，但從提供的「每日報酬」數據來看，在2025年10月8日至10月17日期間，報酬率呈現較大波動。例如，10月9日錄得高達69.32%的顯著漲幅，而10月14日則出現了-25.93%的下跌。這表明即使是在短期內，市場對該標的的情緒變化劇烈。然而，由於原始價格數據的異常，這些報酬率數據亦需審慎解讀，無法作為判斷真實股價波動幅度的依據。本報告後續分析將主要聚焦於基本面和新聞資訊。

### 二、 營運基本面分析

**2.1 營收與獲利概況**

從近五個季度的營收成長率觀察，台積電展現了波動中的增長態勢：
*   **2025年第二季：** 營收成長率為 **11%**。
*   **2025年第一季：** 營收成長率為 **-3%**，是近期唯一出現衰退的季度。
*   **2024年第四季：** 營收成長率為 **14%**。
*   **2024年第三季：** 營收成長率為 **13%**。
*   **2024年第二季：** 營收成長率為 **0%** (持平)。

最新的2025年第二季營收成長率回升至11%，顯示公司營運已擺脫第一季的短暫逆風，重回成長軌道。

獲利方面，最近三季的EPS及其季增率如下：
*   **最新一季EPS：** **15.36元**，季增率達 **10%**。
*   **前一季EPS：** **13.94元**，季增率為 **-4%**。
*   **再前一季EPS：** **14.45元**，季增率達 **15%**。

EPS趨勢呈現先減後增，最新一季EPS及季增率的顯著改善

### 1️⃣1️⃣ 雞蛋水餃股也能做分析

In [12]:
reply = stock_gpt(stock_id="4414")
print(reply)

/tmp/ipython-input-557545184.py:10: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  quarterly_revenue_growth = np.round(stock.quarterly_financials.loc["Total Revenue"].pct_change(-1).dropna().tolist(), 2)
/tmp/ipython-input-557545184.py:16: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  quarterly_eps_growth = np.round(stock.quarterly_financials.loc["Basic EPS"].pct_change(-1).dropna().tolist(), 2)


## 如興 (4414) 近期趨勢分析報告

**報告日期：** 2025年10月18日
**分析標的：** 如興 (4414)

### 報告概述 (Executive Summary)

如興 (4414) 近期股價資訊顯示出極高的異常波動性，與基本面及新聞報導內容存在嚴重數據不一致，使得直接依賴原始股價數據進行分析具有高度風險。然而，綜合其基本面及近期新聞資訊，如興在2025年上半年營運已見改善，成功擺脫全額交割股身份，毛利率與營業利益率皆有顯著提升，並轉虧為盈。然而，近期的月營收年減幅度仍大，且第二季財報顯示稅後淨利仍受業外因素拖累，導致每股盈餘（EPS）大幅虧損。整體而言，如興目前處於營運改善與外部挑戰並存的階段，市場情緒受正面事件（恢復普通交易）與負面因素（營收下滑、淨利虧損）交互影響，投資展望需保持謹慎。

### 一、近期股價表現分析

**1. 數據異常性說明：**
本報告所提供之「近期價格資訊」顯示收盤價位於數十萬至數百萬的極高區間，且每日報酬率在短時間內經歷高達數十%至數百%的劇烈波動（如2025-10-09的81.04%、2025-10-13的178.09%），同時「漲跌價差」卻均顯示為0.0，這與正常的台股交易行為與價格呈現方式嚴重不符，且與新聞報導中提及的股價（如2025-09-05盤中速報的13.4元、漲停價14.0元）存在巨大差異。
**基於此顯著數據異常，本報告將不採信原始「近期價格資訊」中的具體數值進行分析，而是依據新聞報導來推斷其近期股價走勢。**

**2. 依新聞推斷之股價走勢：**
*   **恢復普通交易帶動上漲 (2025年9月上旬)：** 根據2025年9月6日新聞，如興因營運回穩、減資彌補虧損，自9月4日起恢復普通交易方法，帶動股價「周漲2成衝5年高點」。這顯示市場對其交易狀況恢復正常有正面反應，提升了市場信心。
*   **短期劇烈波動 (2025年9月上旬)：** 同樣在9月5日新聞中，如興股價曾出現「急跌-4.32%報13.4元」與「股價拉至漲停，漲停價14.0元」的現象。這表明即使在利好消息刺激下，市場多空拉鋸仍劇烈，股價快速波動是其近期特性。
*   **缺乏10月份實時股價資訊：** 由於原始股價資訊的可靠性問題，缺乏可靠的10月份實時價格數據，無法對10月份具體股價走勢進行精確判斷。但從基本面和月營收